In [1]:
import torch
import json
import json_repair
import pandas as pd

from tqdm import tqdm
from datasets import Dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# from huggingface_hub import login


# login(token = hf_token)

In [3]:
max_seq_length = 4096
model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    model_name="unsloth/Qwen2.5-32B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.4: Fast Qwen2 patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA H100 NVL. Max memory: 93.003 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
model = FastLanguageModel.get_peft_model( 
    model,
    r=32,
    lora_alpha=32,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing=True
)

Unsloth 2024.12.4 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.


## data

In [5]:
df_train = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp_FT/data_files/emorynlp_train_final.csv")
df_test = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp_FT/data_files/emorynlp_test_final.csv")
df_dev = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp_FT/data_files/emorynlp_dev_final.csv")

In [6]:
df_train.iloc[452]

Utterance       Huh, I never really noticed.
Speaker                              ['Bob']
Emotion                              Neutral
Scene_ID                                   9
Utterance_ID                               4
Season                                     1
Episode                                    5
Start_Time                      00:14:11.934
End_Time                        00:14:13.393
Name: 452, dtype: object

In [7]:
df_dev

,Utterance,Speaker,Emotion,Scene_ID,Utterance_ID,Season,Episode,Start_Time,End_Time
0,Coffee.,['Rachel Green'],Neutral,1,1,1,15,00:00:03.795,00:00:05.004
1,Thank you.,['Joey Tribbiani'],Neutral,1,2,1,15,00:00:05.171,00:00:07.423
2,Cappuccino.,['Rachel Green'],Neutral,1,3,1,15,00:00:07.590,00:00:08.757
3,Grazie.,['Ross Geller'],Neutral,1,4,1,15,00:00:08.925,00:00:11.677
4,And a nice hot cider for Monica.,['Rachel Green'],Neutral,1,5,1,15,00:01:27.253,00:01:33.383
...,...,...,...,...,...,...,...,...,...
949,It's better! You can't go to a museum in your ...,['Joey Tribbiani'],Joyful,29,3,4,21,00:21:37.379,00:21:44.135
950,"Well, You could, but... probably just the one ...",['Chandler Bing'],Joyful,29,4,4,21,00:21:45.220,00:21:47.555
951,"I bet we could get videos of all the sites, ge...",['Joey Tribbiani'],Joyful,29,5,4,21,00:21:49.641,00:21:55.604
952,"If we do that, we gotta get Die Hard.",['Chandler Bing'],Joyful,29,6,4,21,00:21:56.189,00:21:58.190


In [8]:
df_dev.head(50)

,Utterance,Speaker,Emotion,Scene_ID,Utterance_ID,Season,Episode,Start_Time,End_Time
0,Coffee.,['Rachel Green'],Neutral,1,1,1,15,00:00:03.795,00:00:05.004
1,Thank you.,['Joey Tribbiani'],Neutral,1,2,1,15,00:00:05.171,00:00:07.423
2,Cappuccino.,['Rachel Green'],Neutral,1,3,1,15,00:00:07.590,00:00:08.757
3,Grazie.,['Ross Geller'],Neutral,1,4,1,15,00:00:08.925,00:00:11.677
4,And a nice hot cider for Monica.,['Rachel Green'],Neutral,1,5,1,15,00:01:27.253,00:01:33.383
5,"Aww, thank you. Uh Rach?",['Monica Geller'],Scared,1,6,1,15,00:01:29.798,00:01:33.383
6,Yeah?,['Rachel Green'],Neutral,1,7,1,15,00:01:34.177,00:01:35.219
7,Why does my cinamon stick have an eraser?,['Monica Geller'],Neutral,1,8,1,15,00:04:11.209,00:04:15.879
8,Oh! That's why. I'm sorry!,['Rachel Green'],Neutral,1,9,1,15,00:04:12.877,00:04:15.879
9,"OK, try this salmon mousse.",['Monica Geller'],Joyful,6,1,1,15,00:04:16.047,00:04:21.218


In [9]:
df_dev.iloc[452]

Utterance             Every year.
Speaker         ['Monica Geller']
Emotion                   Neutral
Scene_ID                        1
Utterance_ID                    8
Season                          3
Episode                         9
Start_Time           00:00:19.436
End_Time             00:00:22.438
Name: 452, dtype: object

### prepare dataset

In [10]:
def build_instruction():
    
    emotion_classes = ["Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"]
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])

    instruction = f"""### You are an expert in Emotion Analysis for the Friends TV show.

INPUT:

- You are given an utterance from an episode of Friends TV show.
- The name of the character who speaks the utterance.

Your task is to classify the utterance with a single emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".

Output Instructions:
1. Return ONLY a JSON object with single key "emotion"
2. ONLY use these emotions: {formatted_classes}
3. Do NOT include any additional text or explanation

RULES:
1. Each utterance must have exactly one emotion from the list above
2. Multiple emotions per utterance are allowed NOT allowed.
3. Maintain exact emotion spelling.
4. No explanations, only JSON output
   
Example format:
{{"emotion": ["Scared"]}}

"""    

    return instruction


In [11]:
def build_response(utterance_emotion):    
                

    return json.dumps({"emotion": [utterance_emotion]})

In [12]:
generation_instruction = build_instruction()

In [33]:
sys_msg_l = []
user_msg_l = []
assistant_msg_l = []

for _, row in df_train.iterrows():
        
        sys_msg = {'role': 'system', 'content': generation_instruction}
        
        # comics_title = row['comics_title']
        # comics_summary = row['summary']
        
        # utterances_l = eval(row['utterance'])
        # pg_utterances = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
        
        # # usr_prompt = f"Comics title: {comics_title}\n" + f"Comics summary: {comics_summary}\n" + "Here is the list of utterances that you will classify: \n" + pg_utterances
        # usr_prompt = build_user_prompt(comics_title, comics_summary, pg_utterances)
        
        user_msg = {'role': 'user', 'content': f"""\n\nUtterance: {row.Utterance}\nSpeaker: {row.Speaker.replace("['", "").replace("']", "")}"""}        
        #assistant_msg = {'role': 'assistant', 'content': ""}
        assistant_msg = {'role': 'assistant', 'content': f"""\n{build_response(row.Emotion)}"""}


        sys_msg_l.append(sys_msg)
        user_msg_l.append(user_msg)
        assistant_msg_l.append(assistant_msg)
        

In [34]:
emory_dataset_train = []

for i in range(len(sys_msg_l)):

    emory_dataset_train.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [35]:
len(emory_dataset_train)

7551

In [36]:
emory_dataset_train[0]

[{'role': 'system',
  'content': '### You are an expert in Emotion Analysis for the Friends TV show.\n\nINPUT:\n\n- You are given an utterance from an episode of Friends TV show.\n- The name of the character who speaks the utterance.\n\nYour task is to classify the utterance with a single emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".\n\nOutput Instructions:\n1. Return ONLY a JSON object with single key "emotion"\n2. ONLY use these emotions: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"\n3. Do NOT include any additional text or explanation\n\nRULES:\n1. Each utterance must have exactly one emotion from the list above\n2. Multiple emotions per utterance are allowed NOT allowed.\n3. Maintain exact emotion spelling.\n4. No explanations, only JSON output\n   \nExample format:\n{"emotion": ["Scared"]}\n\n'},
 {'role': 'user',
  'content': "\n\nUtterance: What you guys don't understand is, for us, kissing is as 

In [37]:
def fix_comics_dataset(comics_dataset):
    fixed_comics_dataset = []
    for conversation in comics_dataset:
        fixed_conversation = []
        for message in conversation:
            if isinstance(message['content'], list):  # If the 'value' is a list of emotions
                message['content'] = ', '.join(message['content'])  # Join the list into a string
            fixed_conversation.append(message)
        fixed_comics_dataset.append(fixed_conversation)
    return fixed_comics_dataset

In [38]:
fixed_comics_dataset = fix_comics_dataset(emory_dataset_train)

In [39]:
dataset = Dataset.from_dict({
    'conversations': fixed_comics_dataset
})

In [40]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 7551
})

In [41]:
dataset[0]['conversations']

[{'content': '### You are an expert in Emotion Analysis for the Friends TV show.\n\nINPUT:\n\n- You are given an utterance from an episode of Friends TV show.\n- The name of the character who speaks the utterance.\n\nYour task is to classify the utterance with a single emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".\n\nOutput Instructions:\n1. Return ONLY a JSON object with single key "emotion"\n2. ONLY use these emotions: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"\n3. Do NOT include any additional text or explanation\n\nRULES:\n1. Each utterance must have exactly one emotion from the list above\n2. Multiple emotions per utterance are allowed NOT allowed.\n3. Maintain exact emotion spelling.\n4. No explanations, only JSON output\n   \nExample format:\n{"emotion": ["Scared"]}\n\n',
  'role': 'system'},
 {'content': "\n\nUtterance: What you guys don't understand is, for us, kissing is as important as any p

In [42]:
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template_comics(examples):
    messages = examples["conversations"]
    #messages = examples['input'] + examples['output']
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


In [43]:
emory_dataset_train = dataset.map(apply_template_comics, batched=True)

Map:   0%|          | 0/7551 [00:00<?, ? examples/s]

In [44]:
emory_dataset_train

Dataset({
    features: ['conversations', 'text'],
    num_rows: 7551
})

In [46]:
print(emory_dataset_train[452]['text'])

<|im_start|>system
### You are an expert in Emotion Analysis for the Friends TV show.

INPUT:

- You are given an utterance from an episode of Friends TV show.
- The name of the character who speaks the utterance.

Your task is to classify the utterance with a single emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".

Output Instructions:
1. Return ONLY a JSON object with single key "emotion"
2. ONLY use these emotions: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"
3. Do NOT include any additional text or explanation

RULES:
1. Each utterance must have exactly one emotion from the list above
2. Multiple emotions per utterance are allowed NOT allowed.
3. Maintain exact emotion spelling.
4. No explanations, only JSON output
   
Example format:
{"emotion": ["Scared"]}

<|im_end|>
<|im_start|>user


Utterance: Huh, I never really noticed.
Speaker: Bob<|im_end|>
<|im_start|>assistant

{"emotion": ["Neutral"]}<|im_e

### Eval dataset

In [47]:
sys_msg_l = []
user_msg_l = []
assistant_msg_l = []

for _, row in df_dev.iterrows():
        
        sys_msg = {'role': 'system', 'content': generation_instruction}
        
        # comics_title = row['comics_title']
        # comics_summary = row['summary']
        
        # utterances_l = eval(row['utterance'])
        # pg_utterances = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
        
        # # usr_prompt = f"Comics title: {comics_title}\n" + f"Comics summary: {comics_summary}\n" + "Here is the list of utterances that you will classify: \n" + pg_utterances
        # usr_prompt = build_user_prompt(comics_title, comics_summary, pg_utterances)
        
        user_msg = {'role': 'user', 'content': f"""\n\nUtterance: {row.Utterance}\nSpeaker: {row.Speaker.replace("['", "").replace("']", "")}"""}       
        #assistant_msg = {'role': 'assistant', 'content': ""}
        assistant_msg = {'role': 'assistant', 'content': f"""\n{build_response(row.Emotion)}"""}


        sys_msg_l.append(sys_msg)
        user_msg_l.append(user_msg)
        assistant_msg_l.append(assistant_msg)
        

In [48]:
emory_dataset_val = []

for i in range(len(sys_msg_l)):

    emory_dataset_val.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [49]:
len(emory_dataset_val)

954

In [50]:
def fix_comics_dataset(comics_dataset):
    fixed_comics_dataset = []
    for conversation in comics_dataset:
        fixed_conversation = []
        for message in conversation:
            if isinstance(message['content'], list):  # If the 'value' is a list of emotions
                message['content'] = ', '.join(message['content'])  # Join the list into a string
            fixed_conversation.append(message)
        fixed_comics_dataset.append(fixed_conversation)
    return fixed_comics_dataset

In [51]:
fixed_comics_dataset = fix_comics_dataset(emory_dataset_val)

In [52]:
dataset = Dataset.from_dict({
    'conversations': fixed_comics_dataset
})

In [53]:
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template_comics(examples):
    messages = examples["conversations"]
    #messages = examples['input'] + examples['output']
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

In [54]:
emory_dataset_val = dataset.map(apply_template_comics, batched=True)

Map:   0%|          | 0/954 [00:00<?, ? examples/s]

In [55]:
emory_dataset_val

Dataset({
    features: ['conversations', 'text'],
    num_rows: 954
})

In [57]:
print(emory_dataset_val[452]['text'])

<|im_start|>system
### You are an expert in Emotion Analysis for the Friends TV show.

INPUT:

- You are given an utterance from an episode of Friends TV show.
- The name of the character who speaks the utterance.

Your task is to classify the utterance with a single emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".

Output Instructions:
1. Return ONLY a JSON object with single key "emotion"
2. ONLY use these emotions: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"
3. Do NOT include any additional text or explanation

RULES:
1. Each utterance must have exactly one emotion from the list above
2. Multiple emotions per utterance are allowed NOT allowed.
3. Maintain exact emotion spelling.
4. No explanations, only JSON output
   
Example format:
{"emotion": ["Scared"]}

<|im_end|>
<|im_start|>user


Utterance: Every year.
Speaker: Monica Geller<|im_end|>
<|im_start|>assistant

{"emotion": ["Neutral"]}<|im_end|>



In [58]:
### train

In [59]:
OUTPUT_DIR = "/Utilisateurs/umushtaq/emotion_analysis_comics/outputs_dir_tmp_emory_ft_native"

In [63]:
args=TrainingArguments(
        do_train=True,
        do_eval=True,
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=128,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        
        eval_strategy="steps",  # Run evaluation during training (can also use "epoch")
        eval_steps=25,  # Perform evaluation every 50 steps
        save_strategy="steps",  # Save the model every few steps
        save_steps=25,  # Save every 200 steps
        load_best_model_at_end=True,
    
        output_dir=OUTPUT_DIR,
        seed=0,
    )

In [64]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=emory_dataset_train,  # Replace with your train dataset
    eval_dataset=emory_dataset_val, 
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=args,
)

Map (num_proc=2):   0%|          | 0/7551 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/954 [00:00<?, ? examples/s]

In [65]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,551 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 128 | Gradient Accumulation steps = 2
\        /    Total batch size = 256 | Total steps = 87
 "-____-"     Number of trainable parameters = 268,435,456


Step,Training Loss,Validation Loss
25,0.332400,0.150305
50,0.285800,0.154411
75,0.236300,0.169464


TrainOutput(global_step=87, training_loss=0.27478627774907255, metrics={'train_runtime': 3954.806, 'train_samples_per_second': 5.728, 'train_steps_per_second': 0.022, 'total_flos': 1.2421469692867154e+18, 'train_loss': 0.27478627774907255, 'epoch': 2.915254237288136})

In [66]:
model = FastLanguageModel.for_inference(model)

In [67]:
#df_test = df[df.split == "TEST"].reset_index(drop=True)

In [68]:
sys_msg_l = []
user_msg_l = []
assistant_msg_l = []

for _, row in df_test.iterrows():
        
        sys_msg = {'role': 'system', 'content': generation_instruction}
        
        # comics_title = row['comics_title']
        # comics_summary = row['summary']
        
        # utterances_l = eval(row['utterance'])
        # pg_utterances = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
        
        # # usr_prompt = f"Comics title: {comics_title}\n" + f"Comics summary: {comics_summary}\n" + "Here is the list of utterances that you will classify: \n" + pg_utterances
        # usr_prompt = build_user_prompt(comics_title, comics_summary, pg_utterances)
        
        user_msg = {'role': 'user', 'content': f"""\n\nUtterance: {row.Utterance}\nSpeaker: {row.Speaker.replace("['", "").replace("']", "")}"""}        
        #assistant_msg = {'role': 'assistant', 'content': ""}
        assistant_msg = {'role': 'assistant', 'content': ""}


        sys_msg_l.append(sys_msg)
        user_msg_l.append(user_msg)
        assistant_msg_l.append(assistant_msg)
        

In [69]:
test_messages = []

for i in range(len(sys_msg_l)):
    
    #obj = {"list_emotion_classes": ["Anger", "Fear"]}

    #comics_dataset.append([human_msg_l[i], assistant_msg_l[i]])
    test_messages.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [70]:
test_messages[0]

[{'role': 'system',
  'content': '### You are an expert in Emotion Analysis for the Friends TV show.\n\nINPUT:\n\n- You are given an utterance from an episode of Friends TV show.\n- The name of the character who speaks the utterance.\n\nYour task is to classify the utterance with a single emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".\n\nOutput Instructions:\n1. Return ONLY a JSON object with single key "emotion"\n2. ONLY use these emotions: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"\n3. Do NOT include any additional text or explanation\n\nRULES:\n1. Each utterance must have exactly one emotion from the list above\n2. Multiple emotions per utterance are allowed NOT allowed.\n3. Maintain exact emotion spelling.\n4. No explanations, only JSON output\n   \nExample format:\n{"emotion": ["Scared"]}\n\n'},
 {'role': 'user',
  'content': "\n\nUtterance: I'm supposed to attach a brackety thing to the side thing

In [71]:
# human_msg_l = []
# assistant_msg_l = []

# for _, row in df_test.iterrows():
        
#         prompt = instruction.replace("<comic_title>", row['comics_title']).replace("<speaker_id>", row['speaker_id']).replace("<utterance>", row['utterance'])
        
#         human_msg = {'role': 'user', 'content': prompt}
        
#         #obj = {"list_emotion_classes": row['emotion_u']}
#         obj = row['emotion_u']
#         assistant_msg = {'role': 'assistant', 'content': ""}
        
#         human_msg_l.append(human_msg)
#         assistant_msg_l.append(assistant_msg)
        
        

In [72]:
# test_messages = []

# for i in range(len(human_msg_l)):
    
#     #obj = {"list_emotion_classes": ["Anger", "Fear"]}

#     test_messages.append([human_msg_l[i], assistant_msg_l[i]])

In [73]:
len(test_messages)

984

In [74]:
# test_messages = test_messages[:100]

In [110]:
raw_outputs = []

for message in tqdm(test_messages):
    
    inputs = tokenizer.apply_chat_template(
    message,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt",
).to("cuda")
    #print(inputs)
    #break
    
    #output = model.generate(input_ids=inputs, max_new_tokens=128)[0]
    output = model.generate(**inputs, max_new_tokens=128)[0]
    
    input_length = inputs.input_ids.shape[1]
    generated_tokens = output[input_length:]
    
    decoded_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)  
    #decoded_output = tokenizer.decode(output, skip_special_tokens=True)
    raw_outputs.append(decoded_output)
    #break

100%|██████████| 984/984 [11:58<00:00,  1.37it/s]


In [111]:
#print(tokenizer.decode(inputs[0]))

In [112]:
len(raw_outputs)

984

In [113]:
raw_outputs

['{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Powerful"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Joyful"]}',
 '{"emotion": ["Joyful"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Scared"]}',
 '{"emotion": ["Mad"]}',
 '{"emotion": ["Mad"]}',
 '{"emotion": ["Scared"]}',
 '{"emotion": ["Scared"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Mad"]}',
 '{"emotion": ["Mad"]}',
 '{"emotion": ["Mad"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Mad"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Joyful"]}',
 '{"emotion": ["Scared"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": ["Neutral"]}',
 '{"emotion": [

In [81]:
preds = []
bad_idx = []

for i, op in enumerate(raw_outputs):
    try:
        preds.append(op.split("\nassistant\n\nassistant\n")[1])
    except:
        print(i)
        bad_idx.append(i)

In [114]:
grounds = df_test.Emotion.tolist()

In [115]:
len(grounds)

984

In [116]:
import json

In [117]:
predictions = [json_repair.loads(e) for e in preds]

In [118]:
predictions

[{'emotion': ['Mad']},
 {'emotion': ['Neutral']},
 {'emotion': ['Powerful']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Powerful']},
 {'emotion': ['Joyful']},
 {'emotion': ['Scared']},
 {'emotion': ['Powerful']},
 {'emotion': ['Neutral']},
 {'emotion': ['Scared']},
 {'emotion': ['Powerful']},
 {'emotion': ['Scared']},
 {'emotion': ['Joyful']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Scared']},
 {'emotion': ['Neutral']},
 {'emotion': ['Scared']},
 {'emotion': ['Mad']},
 {'emotion': ['Scared']},
 {'emotion': ['Neutral']},
 {'emotion': ['Mad']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Joyful']},
 {'emotion': ['Joyful']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Neutral']},
 {'emotion': ['Joyful']},
 {'emotion': ['Peaceful']},
 {'emotion': ['Mad'

In [119]:
# #predictions = [e.split('\n\n')[0] for e in raw_outputs]
# bad_idx = []
# predictions = []

# for idx, e in enumerate(raw_outputs):
#     try:
#         predictions.append(json.loads(e))
#     except:
#         print(idx)
#         bad_idx.append(idx)
        


In [120]:
#len(predictions)

In [121]:
#predictions

In [122]:
#predictions = [json_repair.loads(e) for e in predictions]

In [123]:
#predictions

In [124]:
preds_l = []
bad_idx = []

for i, pred in enumerate(predictions):
    try:        
        preds_l.append(pred['emotion'])
    except:
        print(i)
        bad_idx.append(i)

In [125]:
preds_l

[['Mad'],
 ['Neutral'],
 ['Powerful'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Powerful'],
 ['Joyful'],
 ['Scared'],
 ['Powerful'],
 ['Neutral'],
 ['Scared'],
 ['Powerful'],
 ['Scared'],
 ['Joyful'],
 ['Neutral'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Scared'],
 ['Mad'],
 ['Scared'],
 ['Neutral'],
 ['Mad'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Joyful'],
 ['Joyful'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Joyful'],
 ['Peaceful'],
 ['Mad'],
 ['Joyful'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Powerful'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Scared'],
 ['Joyful'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Neutral'],
 ['Scared'],
 ['Joyful'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Neutral'],
 ['Joyful'],
 ['Neutral'],
 ['Neutral'],
 ['Joyful'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Scared'],
 ['Scared'],
 ['Mad'],
 ['Scared'],
 ['Neutral'],
 ['Neutral'],
 ['

In [126]:
#grounds = [item for i, item in enumerate(grounds) if i not in bad_idx]

In [127]:
len(grounds), len(preds_l)

(984, 984)

In [128]:
preds_l

[['Mad'],
 ['Neutral'],
 ['Powerful'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Powerful'],
 ['Joyful'],
 ['Scared'],
 ['Powerful'],
 ['Neutral'],
 ['Scared'],
 ['Powerful'],
 ['Scared'],
 ['Joyful'],
 ['Neutral'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Scared'],
 ['Mad'],
 ['Scared'],
 ['Neutral'],
 ['Mad'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Joyful'],
 ['Joyful'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Joyful'],
 ['Peaceful'],
 ['Mad'],
 ['Joyful'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Powerful'],
 ['Neutral'],
 ['Neutral'],
 ['Neutral'],
 ['Scared'],
 ['Joyful'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Neutral'],
 ['Scared'],
 ['Joyful'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Neutral'],
 ['Joyful'],
 ['Neutral'],
 ['Neutral'],
 ['Joyful'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Neutral'],
 ['Scared'],
 ['Neutral'],
 ['Scared'],
 ['Scared'],
 ['Mad'],
 ['Scared'],
 ['Neutral'],
 ['Neutral'],
 ['

In [129]:
predictions = [elem[0] for elem in preds_l]

In [130]:
grounds

['Mad',
 'Neutral',
 'Joyful',
 'Neutral',
 'Neutral',
 'Scared',
 'Joyful',
 'Joyful',
 'Sad',
 'Scared',
 'Sad',
 'Sad',
 'Mad',
 'Mad',
 'Mad',
 'Joyful',
 'Joyful',
 'Peaceful',
 'Powerful',
 'Sad',
 'Mad',
 'Mad',
 'Sad',
 'Mad',
 'Scared',
 'Sad',
 'Neutral',
 'Neutral',
 'Joyful',
 'Joyful',
 'Neutral',
 'Joyful',
 'Scared',
 'Joyful',
 'Scared',
 'Joyful',
 'Neutral',
 'Neutral',
 'Joyful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Sad',
 'Powerful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Joyful',
 'Neutral',
 'Scared',
 'Neutral',
 'Joyful',
 'Scared',
 'Powerful',
 'Neutral',
 'Joyful',
 'Joyful',
 'Scared',
 'Joyful',
 'Neutral',
 'Joyful',
 'Joyful',
 'Sad',
 'Joyful',
 'Peaceful',
 'Neutral',
 'Neutral',
 'Scared',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Scared',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Joyful',
 'Peaceful',
 'Joyful',
 'Joyful',
 'Joyful',
 'Joyful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Joyful',
 'Neutral',
 'Joyful',
 'Joyful',

In [131]:
grounds

['Mad',
 'Neutral',
 'Joyful',
 'Neutral',
 'Neutral',
 'Scared',
 'Joyful',
 'Joyful',
 'Sad',
 'Scared',
 'Sad',
 'Sad',
 'Mad',
 'Mad',
 'Mad',
 'Joyful',
 'Joyful',
 'Peaceful',
 'Powerful',
 'Sad',
 'Mad',
 'Mad',
 'Sad',
 'Mad',
 'Scared',
 'Sad',
 'Neutral',
 'Neutral',
 'Joyful',
 'Joyful',
 'Neutral',
 'Joyful',
 'Scared',
 'Joyful',
 'Scared',
 'Joyful',
 'Neutral',
 'Neutral',
 'Joyful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Sad',
 'Powerful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Joyful',
 'Neutral',
 'Scared',
 'Neutral',
 'Joyful',
 'Scared',
 'Powerful',
 'Neutral',
 'Joyful',
 'Joyful',
 'Scared',
 'Joyful',
 'Neutral',
 'Joyful',
 'Joyful',
 'Sad',
 'Joyful',
 'Peaceful',
 'Neutral',
 'Neutral',
 'Scared',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Scared',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Joyful',
 'Peaceful',
 'Joyful',
 'Joyful',
 'Joyful',
 'Joyful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Joyful',
 'Neutral',
 'Joyful',
 'Joyful',

In [132]:
predictions

['Mad',
 'Neutral',
 'Powerful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Powerful',
 'Joyful',
 'Scared',
 'Powerful',
 'Neutral',
 'Scared',
 'Powerful',
 'Scared',
 'Joyful',
 'Neutral',
 'Neutral',
 'Scared',
 'Neutral',
 'Scared',
 'Mad',
 'Scared',
 'Neutral',
 'Mad',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Joyful',
 'Joyful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Joyful',
 'Peaceful',
 'Mad',
 'Joyful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Powerful',
 'Neutral',
 'Neutral',
 'Neutral',
 'Scared',
 'Joyful',
 'Neutral',
 'Scared',
 'Neutral',
 'Neutral',
 'Scared',
 'Joyful',
 'Neutral',
 'Scared',
 'Neutral',
 'Neutral',
 'Joyful',
 'Neutral',
 'Neutral',
 'Joyful',
 'Neutral',
 'Scared',
 'Neutral',
 'Neutral',
 'Scared',
 'Neutral',
 'Scared',
 'Scared',
 'Mad',
 'Scared',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Joyful',
 'Neutral',
 'Scared',
 'Scared',
 'Joyful',
 'Scared',
 'Neutral',
 'Neutral',
 'Mad',
 'Sc

In [133]:
# bad_idx = []

# for idx, (i,j) in enumerate(zip(grounds, preds_l)):
#     if len(i) != len(j):
#         print(idx, len(i), len(j))
#         bad_idx.append(idx)

In [134]:
# bad_idx.sort(reverse=True)

# # Remove elements from 'grounds' at the specified indices
# for idx in bad_idx:
    
#     del grounds[idx]
#     del preds_l[idx]

In [135]:
# grounds = [item for sublist in grounds for item in sublist]
# predictions = [item for sublist in preds_l for item in sublist]

In [136]:
len(grounds), len(predictions)

(984, 984)

In [137]:
#mlb = MultiLabelBinarizer()

In [138]:
# y_true_mhot = mlb.fit_transform(grounds)
# y_pred_mhot = mlb.transform(predictions)

In [139]:
#y_pred_mhot.shape

In [140]:
#y_pred_mhot.shape

In [141]:
#mlb.classes_

In [142]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

      Joyful      0.528     0.470     0.498       217
         Mad      0.371     0.419     0.393        86
     Neutral      0.405     0.743     0.524       288
    Peaceful      0.286     0.018     0.034       111
    Powerful      0.179     0.052     0.081        96
         Sad      0.500     0.014     0.028        70
      Scared      0.234     0.259     0.246       116

    accuracy                          0.396       984
   macro avg      0.358     0.282     0.258       984
weighted avg      0.380     0.396     0.340       984

